In [1]:
pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/1b/cc/0af9c07f8d714ea563b12383a7e5bde9479cf32413ee2f346a9c5a801f22/pandas-2.3.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/5c/23/c7abc0ca0a1526a0774eca151daeb8de62ec457e77262b66b359c3c7679e/tzdata-2025.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 4.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 3.4 MB/s eta 0:00:00a 0:00:01

[notice] A new

In [ ]:
import random
import time
import matplotlib.pyplot as plt
import pandas as pd

from trader import Trader
from stock import Stock
from exchange import Exchange
from share import Share
from order import Order, BuyOrder, SellOrder
from market_maker import MarketMaker

money_data = pd.DataFrame(columns=["Trader", "Time", "Money"])
stock_price_data = pd.DataFrame(columns=["Time", "REL", "AIR", "JIO"])

stock_exchange = Exchange()
stock1 = Stock(stock_exchange, 12.90, "REL")
stock2 = Stock(stock_exchange, 11.40, "AIR")
stock3 = Stock(stock_exchange, 10.80, "JIO")
stock_exchange.stocks = {"REL": stock1, "AIR": stock2, "JIO": stock3}

# mm = MarketMaker(11, random.randint(20000, 50000), random.randint(20000, 50000), {}, stock_exchange)

for i in range(11):
    trader = Trader(i, random.randint(20000, 50000), random.randint(20000, 50000), {}, stock_exchange)
    if i == 10:
        trader = MarketMaker(i, random.randint(20000, 50000), random.randint(20000, 50000), {}, stock_exchange)
    for _ in range(random.randint(1000, 20000)):
        stock = random.choice([stock1, stock2, stock3])
        share = Share(stock, i)
        trader.portfolio.setdefault(stock.ticker, []).append(share)
    stock_exchange.traders[i] = trader

for j in range(100):
    for i in range(10):
        stock = random.choice([stock1, stock2, stock3])
        order_type = random.choice(["BID", "OFFER"])
        price = round(random.uniform(stock.cur_price - 2, stock.cur_price + 2), 2)
        price = max(price, 0)
        quantity = random.randint(10, 40)
        timestamp = time.time()

        stock_exchange.traders[i].place_limit_order(
            type=order_type, quantity=quantity,
            ticker=stock.ticker, price=price, time=timestamp
        )

        money = stock_exchange.traders[i].profit()

        money_data = pd.concat([money_data, pd.DataFrame([{
            "Trader": i,
            "Time": j,
            "Money": money
        }])], ignore_index=True)

        stock = random.choice([stock1, stock2, stock3])
        quantity = random.randint(10, 40)
        timestamp = time.time()



        stock_exchange.traders[10].place_order(
            quantity=quantity,
            ticker=stock.ticker, time=timestamp
        )

        money = stock_exchange.traders[10].profit()

    money_data = pd.concat([money_data, pd.DataFrame([{
            "Trader": 10,
            "Time": j,
            "Money": money
        }])], ignore_index=True)


    stock_price_data = pd.concat([stock_price_data, pd.DataFrame([{
        "Time": j,
        "REL": stock1.cur_price,
        "AIR": stock2.cur_price,
        "JIO": stock3.cur_price
    }])], ignore_index=True)

# Plot balance for each trader
for i in range(11):
    trader_data = money_data[money_data["Trader"] == i]
    plt.figure(figsize=(12, 4))
    plt.plot(trader_data["Time"], trader_data["Money"], marker='o')
    plt.title(f"Trader {i} - Money Over Time")
    plt.xlabel("Time")
    plt.ylabel("Trading Balance")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Plot stock prices over time
for ticker in ["REL", "AIR", "JIO"]:
    plt.figure(figsize=(12, 4))
    plt.plot(stock_price_data["Time"], stock_price_data[ticker], label=ticker, marker='o')
    plt.title(f"{ticker} - Price Over Time")
    plt.xlabel("Time")
    plt.ylabel("Price")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
